In [0]:
from pyspark.sql.functions import current_date

# Incoming updates (source)
source = spark.createDataFrame([
    (1, "Alice", "HR"),
    (2, "Bob", "Finance"),
    (3, "Charlie", "Sales")  # New record
], ["emp_id", "name", "department"])

source.createOrReplaceTempView("source_df")


In [0]:
from delta.tables import DeltaTable

# Initial load
initial = spark.createDataFrame([
    (1, "Alice", "IT"),
    (2, "Bob", "Finance")
], ["emp_id", "name", "department"])

initial.write.format("delta").mode("overwrite").saveAsTable("bronze.scd_demo")


✅ **SCD Type 1: Overwrite Without History**

In [0]:
# Overwrite changes directly (no history)
target = DeltaTable.forName(spark, "bronze.scd_demo")

target.alias("target") \
  .merge(
    source.alias("source"),
    "target.emp_id = source.emp_id"
  ) \
  .whenMatchedUpdateAll() \
  .whenNotMatchedInsertAll() \
  .execute()


In [0]:
spark.sql("select * from bronze.scd_demo").show()

**Step 2: Apply SCD2 Logic**

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType, BooleanType
from datetime import date

schema = StructType([
    StructField("emp_id", IntegerType(), False),
    StructField("name", StringType(), False),
    StructField("department", StringType(), False),
    StructField("start_date", DateType(), False),
    StructField("end_date", DateType(), True),
    StructField("is_current", BooleanType(), False)
])

initial_scd2 = spark.createDataFrame([
    (1, "Alice", "IT", date(2023, 1, 1), None, True),
    (2, "Bob", "Finance", date(2023, 1, 1), None, True)
], schema)

initial_scd2.write.format("delta").mode("overwrite").partitionBy("emp_id").saveAsTable("bronze.scd2_demo")

In [0]:
from pyspark.sql.functions import lit

today = current_date()

# Load tables
target = DeltaTable.forName(spark, "bronze.scd2_demo")

# Step 2a: Close current records if changed
target.alias("t").merge(
    source.alias("s"),
    "t.emp_id = s.emp_id AND t.is_current = true AND (t.name <> s.name OR t.department <> s.department)"
).whenMatchedUpdate(set={
    "end_date": today,
    "is_current": lit(False)
}).execute()

# Step 2b: Insert new records (new or updated)
new_data = source.withColumn("start_date", today) \
                 .withColumn("end_date", lit(None)) \
                 .withColumn("is_current", lit(True))

target.alias("t").merge(
    new_data.alias("s"),
    "t.emp_id = s.emp_id AND t.is_current = true"
).whenNotMatchedInsertAll().execute()


In [0]:
%sql
select * from bronze.scd2_demo

In [0]:
import pandas as pd
url = "https://raw.githubusercontent.com/richhuwtaylor/adventure-works/main/data/AdventureWorks%20Customer%20Lookup.csv"

df_pd = pd.read_csv(url, encoding='ISO-8859-1')
df = spark.createDataFrame(df_pd)
df.write.format("delta").mode("overwrite").saveAsTable("bronze.customer_lookup")

In [0]:
from pyspark.sql.functions import col, lit
df = df.withColumn("total_children", lit("REDACTED"))

# Row-level security: Filter rows based on 'marital_status'
df1 = df.filter(col("gender") == "M")

# Display the DataFrame
display(df1)

In [0]:
display(df)

In [0]:
%sql
CREATE OR REPLACE FUNCTION adb_practice.bronze.mask_email(email STRING)
RETURNS STRING
RETURN
  CASE
    WHEN is_member('email_access_group') THEN email
    ELSE '*****@****.com'
  END;

In [0]:
%sql
select * from bronze.customer_lookup

**Lineage**

In [0]:
%sql
CREATE OR REPLACE TABLE bronze.customers (
  customer_id INT,
  name STRING,
  region STRING
);

INSERT INTO bronze.customers VALUES
  (1, 'Alice', 'North'),
  (2, 'Bob', 'South'),
  (3, 'Charlie', 'East');


In [0]:
%sql
CREATE OR REPLACE TABLE bronze.orders (
  order_id INT,
  customer_id INT,
  amount DOUBLE
);

INSERT INTO bronze.orders VALUES
  (101, 1, 250.0),
  (102, 2, 300.0),
  (103, 1, 150.0),
  (104, 3, 400.0);


In [0]:
%sql
CREATE OR REPLACE TABLE bronze.customer_orders_summary AS
SELECT
  c.customer_id,
  c.name,
  c.region,
  COUNT(o.order_id) AS total_orders,
  SUM(o.amount) AS total_spent
FROM bronze.customers c
JOIN bronze.orders o
  ON c.customer_id = o.customer_id
GROUP BY c.customer_id, c.name, c.region;


In [0]:
%sql
select * from bronze.customer_orders_summary